# Implementing a Recurrent Neural Network (RNN) from Scratch in Python

### Overview

Recurrent Neural Networks (RNNs) are a class of neural networks that excel in processing sequential data. They are a fundamental building block in the field of natural language processing (NLP) and are also widely used in other applications such as time-series analysis, speech recognition, and music generation. The ability of RNNs to maintain a 'memory' of previous inputs makes them uniquely suited for tasks where context and order are crucial.

### Goal

In this notebook, I manually implement a basic RNN using only NumPy to deepen my understanding of its mechanics. This will serve as an exercise to solidify my theoretical knowledge and also improve my practical skills in building neural networks from the ground up. Throughout this project, I will include detailed markdown cells explaining different components of the architecture so that this notebook can serve as a comprehensive reference point in the future.


In [5]:
#Basic Scientific Libraries
import numpy as np
import pandas as pd

#NLP Libraries for Data Preprocessing
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

## Gentle Introduction 

### Understanding Sequential Data

**Sequential data** refers to any data where the order matters. This can be anything from a sentence in a text (where the sequence of words affects the meaning) to a series of stock prices (where past prices can influence future prices). 

To illustrate handling sequential data in machine learning, specifically with Recurrent Neural Networks (RNNs), consider a sentence containing $n$ words. In an RNN model, this sentence can be represented as a sequence of $n$ word embeddings:

- $X_{0}$ represents the word embedding of the first word,
- $X_{1}$ represents the word embedding of the second word,
- ...,
- $X_{n-1}$ represents the word embedding of the $n$th word.

It's important to note the zero-based indexing used here: $X_{0}$ is the first word's embedding, $X_{1}$ the second, and so on up to $X_{n-1}$, which is the $n$th word's embedding.

### How RNNs Leverage Sequential Data

RNNs are particularly well-suited for processing this type of data because they maintain an internal state that gets updated as they process each word in a sequence. This mechanism allows them to consider not just the current word's information (embedded in $X_t$), but also the context provided by all previous words in the sequence.

For instance, when predicting the next word in a sentence after the second word $X_1$, the RNN uses both the information in $X_1$ and the context accumulated from processing the first word $X_0$. This ability to carry information across different time steps enables RNNs to make more informed predictions and capture dependencies that span across the sequence. Let's take a look at what this looks like mathematically:

1. **State Update Equation**:
   At each step in the sequence, the RNN updates its state by combining information from the current input and the previous state. This updated state is computed using:
   $$
   h_t = \tanh(W_{hh} h_{t-1} + W_{xh} x_t + b_h)
   $$
   where:
   - $h_{t-1}$ is the previous state,
   - $x_t$ is the current input,
   - $W_{hh}$ and $W_{xh}$ are weights that determine how the previous state and the current input respectively influence the new state,
   - $b_h$ is a bias term,
   - $\tanh$ is a nonlinear function that helps model complex patterns.

2. **Output Generation**:
   The output at each step is generated based on the current state:
   $$
   o_t = W_{hy} h_t + b_y
   $$
   where $W_{hy}$ is the weight matrix connecting the state to the output and $b_y$ is the output bias.


We shall implement these functions in Python below.

In [1]:
def tanh(x):
    return (np.exp(x) - np.exp(-x))/(np.exp(x)+np.exp(-x))

In [4]:
def update_hidden_state(h_prev, x_t, W_hh, W_xh, b_h):
    """
    Update the hidden state for a recurrent neural network.

    Parameters:
    h_prev (numpy.ndarray): The previous hidden state (vector), shape (hidden_size, 1)
    x_t (numpy.ndarray): The current input (vector), shape (input_size, 1)
    W_hh (numpy.ndarray): Weight matrix for the hidden-to-hidden connection, shape (hidden_size, hidden_size)
    W_xh (numpy.ndarray): Weight matrix for the input-to-hidden connection, shape (hidden_size, input_size)
    b_h (numpy.ndarray): Bias vector for the hidden state, shape (hidden_size, 1)

    Returns:
    numpy.ndarray: The updated hidden state (vector), shape (hidden_size, 1)
    """
    z = np.dot(W_hh, h_prev) + np.dot(W_xh, x_t) + b_h
    h_t = tanh(z)
    return h_t

In [2]:
def calculate_step_output(W_hy, h_t, b_y):
    return np.dot(W_hy, h_t) + b_y

### Further Intuition

Before we continue on, let's break down what's going on in the state update equation. The $W_{hh} h_{t-1}$ term, which is the linear combination of the weights between hidden states and the previous state, tells us how much the previous state contributes to our current output.

1. **$W_{hh} h_{t-1}$**: This term represents the influence of the previous hidden state on the current state. Here:
   - $W_{hh}$ is a weight matrix that captures the relationship between the previous hidden state and the current one. It is crucial for the network to remember or carry forward information from the past.
   - $h_{t-1}$ is the hidden state from the previous timestep. It holds information that the network has processed up to that point.
   
   The product $W_{hh} h_{t-1}$ computes how much of the past information (from the previous timestep) should be retained or modified. It can be seen as the network's memory influencing its current decision or state.

2. **$W_{xh} x_t$**: This term deals with the current input:
   - $W_{xh}$ is a weight matrix that connects the input layer to the hidden layer.
   - $x_t$ is the input at the current timestep.

   The product $W_{xh} x_t$ represents how the current input contributes to the new state. It's crucial for integrating new information into the network.

3. **$b_h$**: This is the bias term for the hidden layer. It provides additional flexibility to the model, allowing it to better fit the data by adjusting the output independently of the weighted inputs. Biases help to shift the activation function to either the left or right, which can be critical for learning patterns.

4. **Activation Function $\tanh$**: The hyperbolic tangent function, $\tanh$, is used to add non-linearity to the process, helping the network learn complex patterns. It also squashes the output to be between -1 and 1, which helps in stabilizing the network by controlling the scale of output values.

By combining these components, the RNN can maintain a balance between remembering past information and incorporating new inputs, thereby effectively handling sequential data. This equation is repeated at each timestep, continuously updating the hidden state throughout the sequence.

A **state** in the context of an RNN represents the memory of the network, encapsulating information about the inputs it has processed so far. In simple terms, the state helps the network 'remember' what it has seen in the sequence, which influences how it processes and reacts to new inputs.

### Components of an RNN

1. **Input Layer**: This is where the network receives the sequence of data one element at a time.
2. **Hidden Layer (Recurrent Layer)**: This is the core of an RNN, where the inputs and previous states are combined to update the current state.
3. **Output Layer**: Depending on the application, the network can produce an output at each step or only at the end of the sequence.

2. **Output Generation**:
   The output at each step is generated based on the current state:
   $$
   o_t = W_{hy} h_t + b_y
   $$
   where $W_{hy}$ is the weight matrix connecting the state to the output and $b_y$ is the output bias.

### Learning

The parameters of the model (i.e., $W_{hh}$, $W_{xh}$, $W_{hy}$, $b_h$, and $b_y$) are learned through a process called backpropagation through time (BPTT). BPTT is an extension of the standard backpropagation algorithm used in feedforward networks, modified to handle the sequential nature of RNNs. It involves:

- **Forward Pass**: Computing the predicted output and the hidden states for all time steps.
- **Loss Calculation**: Evaluating the discrepancy between the predicted output and the actual target output using a loss function, typically Mean Squared Error (MSE) or Cross-Entropy Loss.
- **Backward Pass**: Propagating the error back through the network to update the weights and biases to minimize the loss.

### Challenges

Despite their flexibility and power, RNNs are often challenged by issues like:
- **Vanishing Gradient Problem**: During backpropagation, gradients can vanish (become very small) as they are propagated back through time and layers, making learning long-term dependencies practically impossible.
- **Exploding Gradient Problem**: Conversely, gradients can also explode (become very large), which may lead to diverging weights during training.

To address these issues, more sophisticated variants of RNNs, such as Long Short-Term Memory (LSTM) networks and Gated Recurrent Units (GRUs), have been developed.

This section has laid out the fundamental architecture of RNNs and their mathematical underpinnings, providing a foundation for understanding how they operate and are implemented in practical applications.

In [1]:
h_prev = np.zeros((300, 1))
for i, embedding in enumerate(sequence):
    
    x_t = embedding
    hidden_state = update_hidden_state()

NameError: name 'np' is not defined

In [4]:
data = pd.read_csv('datasets/movie_reviews.csv')
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


First, we need to preprocess the text. This includes:
- Tokenization
- Lowercasing all the words
- Removing punctuation and special

In [7]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Return the preprocessed tokens
    return filtered_tokens

# Apply preprocessing to each review
data['processed_reviews'] = data['review'].apply(preprocess_text)
data

,review,sentiment,processed_reviews
0,One of the other reviewers has mentioned that ...,positive,"[one, reviewers, mentioned, watching, oz, epis..."
1,A wonderful little production. <br /><br />The...,positive,"[wonderful, little, production, br, br, filmin..."
2,I thought this was a wonderful way to spend ti...,positive,"[thought, wonderful, way, spend, time, hot, su..."
3,Basically there's a family where a little boy ...,negative,"[basically, theres, family, little, boy, jake,..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[petter, matteis, love, time, money, visually,..."
...,...,...,...
49995,I thought this movie did a down right good job...,positive,"[thought, movie, right, good, job, wasnt, crea..."
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,I am a Catholic taught in parochial elementary...,negative,"[catholic, taught, parochial, elementary, scho..."
49998,I'm going to have to disagree with the previou...,negative,"[im, going, disagree, previous, comment, side,..."
